In [149]:
import os
import subprocess
import sys
import time

TRAINING_APP_FOLDER = 'training_app'
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)


In [150]:
REGION = 'us-central1'
#ENDPOINT = '<YOUR_ENDPOINT>' e.g. '337dd39580cbcbd2-dot-us-central2.pipelines.googleusercontent.com'
ARTIFACT_STORE_URI = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [151]:
# Create starified samples of training, validation, incremental data
import time
training_dataset_path = 'gs://input_data_amy_bkt1/input_data/Anonymized_Fermentation_Data_final.xlsx'
validation_dataset_path = 'gs://input_data_amy_bkt1/input_data/Anonymized_Fermentation_Validation_Data_final.xlsx'

## Step 2: Write the training script

In [152]:
%%writefile {TRAINING_APP_FOLDER}/train.py

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import subprocess
import sys

import fire
import pickle
import numpy as np
import pandas as pd

import json

import pickle
import time


import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

from sklearn import linear_model
from sklearn import model_selection
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from googleapiclient import discovery
from googleapiclient import errors




def train_evaluate(job_dir, training_dataset_path, validation_dataset_path, alpha, max_iter, hptune):
    
    #data = pd.read_excel(bkt_excl,sheet_name='data') 
    df_train = pd.read_excel(training_dataset_path,sheet_name='data')
    df_validation = pd.read_excel(validation_dataset_path,sheet_name='data')
    meta_data = pd.read_excel(training_dataset_path,sheet_name='meta data') 
    
    #df_train = pd.read_csv(training_dataset_path)
    #df_validation = pd.read_csv(validation_dataset_path)

    if not hptune:
        df_train = pd.concat([df_train, df_validation])

    #numeric_feature_indexes = slice(0, 10)
    #categorical_feature_indexes = slice(10, 12)
   
    
    #Prepare data for analysis
    #Split out numeric from categorical varibles

    numeric_vars = ((df_train.dtypes == 'float64') | (df_train.dtypes == 'int64')) & (meta_data['variable type'] == 'independent').values
    numeric_x_data = df_train[df_train.columns[numeric_vars]]

    numeric_vars_val = ((df_validation.dtypes == 'float64') | (df_validation.dtypes == 'int64')) & (meta_data['variable type'] == 'independent').values
    numeric_x_data_val = df_validation[df_validation.columns[numeric_vars_val]]

    
    cat_vars = ((df_train.dtypes == 'string') | (df_train.dtypes == 'object')) & (meta_data['variable type'] == 'independent').values
    cat_x_data = df_train[df_train.columns[cat_vars]]

    #Things to try to predict

    y_data = df_train[df_train.columns[(meta_data['target'] == 1).values]]
    y_data_val = df_validation[df_validation.columns[(meta_data['target'] == 1).values]]
    
   
    # meta data about variables

    meta_data = meta_data.set_index('name')    

    #Impute missing with median #handle missing values with median. SKlearn provides imputer
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')

    #auto scale
    scaler = StandardScaler()

    #pipe = Pipeline([('imputer',imputer),('scaler', scaler)])
    
    #preprocessor = ColumnTransformer(
    #transformers=[
    #    ('num', StandardScaler(), numeric_x_data),
    #    ('cat', OneHotEncoder(), cat_x_data) 
    #])

    pipe = Pipeline([
        ('imputer',imputer), ('scaler', scaler) 
    ])

    
    scaled_numeric_x_data = pipe.fit_transform(numeric_x_data)
    scaled_numeric_x_data_val = pipe.fit_transform(numeric_x_data_val)
    
    pca = PCA(n_components=3)
    pca_result = pca.fit_transform(scaled_numeric_x_data)
    pca_result_val = pca.fit_transform(scaled_numeric_x_data_val)

    
    #pipe = Pipeline([
    #    ('preprocessor', preprocessor),('imputer',imputer),
    #    ('sgdregressor', SGDRegressor(loss='squared_loss',tol=1e-3))
    #])

    
    pipeline = Pipeline([
        ('sgdregressor', SGDRegressor(loss='squared_loss',tol=1e-3))
    ])
    
    
    ##########
    #preprocessor = ColumnTransformer(
    #transformers=[
    #    ('num', StandardScaler(), numeric_feature_indexes),
    #    ('cat', OneHotEncoder(), categorical_feature_indexes) 
    #])

    #pipeline = Pipeline([
    #    ('preprocessor', preprocessor),
    #    ('classifier', SGDClassifier(loss='log',tol=1e-3))
    #])

    #num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}
    #df_train = df_train.astype(num_features_type_map)
    #df_validation = df_validation.astype(num_features_type_map) 
     
    
    print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))
    df_train = get_results(pca_result,'pca-', add = y_data)
    df1_train = df_train
    df2_train = df1_train.dropna()
    
    df_validation = get_results(pca_result_val,'pca-', add = y_data_val)
    df1_validation = df_validation
    df2_validation = df1_validation.dropna()

    
    #X_train = df_train.drop("Run_Execution","Run_Performance","Product_Produced__g","Titer_End__g_over_kg", axis=1)
    #y_train = df_train["Product_Produced__g"]
    #X_train = df2_train.drop("Run_Execution","Run_Performance","Product_Produced__g","Titer_End__g_over_kg", axis=1)
    X_train = df2_train.drop(columns=["Run_Execution","Run_Performance","Product_Produced__g","Titer_End__g_over_kg"])
    y_train = df2_train["Product_Produced__g"]


    #pipeline.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter)
    pipeline.set_params(sgdregressor__alpha=alpha, sgdregressor__max_iter=max_iter)   
    pipeline.fit(X_train, y_train)

    if hptune:
        X_validation = df2_validation.drop(columns=["Run_Execution","Run_Performance","Product_Produced__g","Titer_End__g_over_kg"])
        y_validation = df2_validation["Product_Produced__g"]
        accuracy = pipeline.score(X_validation, y_validation)
        print('Model accuracy: {}'.format(accuracy))
        # Log it with hypertune
        hpt = hypertune.HyperTune()
        hpt.report_hyperparameter_tuning_metric(
          hyperparameter_metric_tag='accuracy',
          metric_value=accuracy
        )

    # Save the model
    if not hptune:
        model_filename = 'model.pkl'
        with open(model_filename, 'wb') as model_file:
            pickle.dump(pipeline, model_file)
        gcs_model_path = "{}/{}".format(job_dir, model_filename)
        subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
        print("Saved model in: {}".format(gcs_model_path)) 

        

def get_results(res,prefix='',ncol=3, add=None):
    #collect results    
    out= pd.DataFrame()
    for i in range(ncol):
        key = prefix + str(i+1)
        value = res[:,i]
        out.loc[:,key] = value
    
    if add is not None:
        out = pd.concat([out,add],axis=1)
    
    return out

        
if __name__ == "__main__":
    fire.Fire(train_evaluate)





Overwriting training_app/train.py


## Step 3: Package the script into a docker image.

In [153]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]


Overwriting training_app/Dockerfile


In [154]:
!cat base_image/Dockerfile

cat: base_image/Dockerfile: No such file or directory


In [155]:
!cat trainer_image/Dockerfile

cat: trainer_image/Dockerfile: No such file or directory


## Step 4: Build the docker image

In [156]:
IMAGE_NAME='trainer_image'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)

In [157]:
!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

Creating temporary tarball archive of 5 file(s) totalling 12.5 KiB before compression.
Uploading tarball of [training_app] to [gs://etl-project-datahub_cloudbuild/source/1599064124.944803-638cf3bf1baf4081984c065aab452fed.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/etl-project-datahub/builds/c667b57a-f1e0-4031-85d0-490bb0cb5149].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/c667b57a-f1e0-4031-85d0-490bb0cb5149?project=448067079266].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "c667b57a-f1e0-4031-85d0-490bb0cb5149"

FETCHSOURCE
Fetching storage object: gs://etl-project-datahub_cloudbuild/source/1599064124.944803-638cf3bf1baf4081984c065aab452fed.tgz#1599064125372248
Copying gs://etl-project-datahub_cloudbuild/source/1599064124.944803-638cf3bf1baf4081984c065aab452fed.tgz#1599064125372248...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed ove

# Run hyperparameter tuning jobs

### Step 1: Create the hyperparameter configuration file

In [158]:
%%writefile {TRAINING_APP_FOLDER}/hptuning_config.yaml

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 3
    maxParallelTrials: 3
    hyperparameterMetricTag: accuracy
    enableTrialEarlyStopping: TRUE 
    params:
    - parameterName: max_iter
      type: DISCRETE
      discreteValues: [
          200,
          500
          ]
    - parameterName: alpha
      type: DOUBLE
      minValue:  0.00001
      maxValue:  0.001
      scaleType: UNIT_LINEAR_SCALE


Overwriting training_app/hptuning_config.yaml


### Step 2: Start the hyperparameter tuning job

job_dir -> GCS Path for storing the job packages & model.

training_dataset_path -> GCS path holding training dataset.

validation_dataset_path -> GCS path holding validation dataset.

alpha -> hyperparameter

max_iter -> hyperparameter

hptune -> variable to decide if hyperparameter tuning is to be done or not.


In [159]:
GCS_STAGING_PATH = '{}/staging'.format(ARTIFACT_STORE_URI)
gcs_root= GCS_STAGING_PATH 
print(gcs_root)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE_URI)
print(JOB_DIR_ROOT)
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
print(JOB_NAME)
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
print(JOB_DIR)

gs://workshop_trial_artifact_store_pp/staging
gs://workshop_trial_artifact_store_pp/jobs
JOB_20200902_163433
gs://workshop_trial_artifact_store_pp/jobs/JOB_20200902_163433


In [160]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'dataset.xlsx')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'dataset.xlsx')

In [161]:
print(JOB_DIR_ROOT)
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
print(JOB_NAME)
print(JOB_DIR)

gs://workshop_trial_artifact_store_pp/jobs
JOB_20200902_163437
gs://workshop_trial_artifact_store_pp/jobs/JOB_20200902_163437


In [162]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
--config $TRAINING_APP_FOLDER/hptuning_config.yaml \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--hptune


Job [JOB_20200902_163440] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20200902_163440

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20200902_163440
jobId: JOB_20200902_163440
state: QUEUED


In [143]:
!gcloud ai-platform jobs describe $JOB_NAME

createTime: '2020-09-02T15:45:42Z'
etag: Ay31Bn19aeg=
jobId: JOB_20200902_154540
startTime: '2020-09-02T15:45:43Z'
state: RUNNING
trainingInput:
  args:
  - --training_dataset_path=gs://workshop_trial_artifact_store_pp/data/training/dataset.xlsx
  - --validation_dataset_path=gs://workshop_trial_artifact_store_pp/data/validation/dataset.xlsx
  - --hptune
  hyperparameters:
    enableTrialEarlyStopping: true
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxParallelTrials: 1
    maxTrials: 1
    params:
    - discreteValues:
      - 200.0
      - 500.0
      parameterName: max_iter
      type: DISCRETE
    - maxValue: 0.001
      minValue: 1e-05
      parameterName: alpha
      scaleType: UNIT_LINEAR_SCALE
      type: DOUBLE
  jobDir: gs://workshop_trial_artifact_store_pp/jobs/JOB_20200902_154540
  masterConfig:
    imageUri: gcr.io/etl-project-datahub/trainer_image:latest
  region: us-central1
trainingOutput:
  isHyperparameterTuningJob: true

View job in the Cloud Console

In [144]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

^C


Command killed by keyboard interrupt



In [147]:
from googleapiclient import discovery
from googleapiclient import errors
ml = discovery.build('ml', 'v1')

job_id = 'projects/{}/jobs/{}'.format(PROJECT_ID, JOB_NAME)
request = ml.projects().jobs().get(name=job_id)

try:
    response = request.execute()
except errors.HttpError as err:
    print(err)
except:
    print("Unexpected error")
    
response


{'jobId': 'JOB_20200902_154540',
 'trainingInput': {'args': ['--training_dataset_path=gs://workshop_trial_artifact_store_pp/data/training/dataset.xlsx',
   '--validation_dataset_path=gs://workshop_trial_artifact_store_pp/data/validation/dataset.xlsx',
   '--hptune'],
  'hyperparameters': {'goal': 'MAXIMIZE',
   'params': [{'parameterName': 'max_iter',
     'type': 'DISCRETE',
     'discreteValues': [200, 500]},
    {'parameterName': 'alpha',
     'minValue': 1e-05,
     'maxValue': 0.001,
     'type': 'DOUBLE',
     'scaleType': 'UNIT_LINEAR_SCALE'}],
   'maxTrials': 1,
   'maxParallelTrials': 1,
   'hyperparameterMetricTag': 'accuracy',
   'enableTrialEarlyStopping': True},
  'region': 'us-central1',
  'jobDir': 'gs://workshop_trial_artifact_store_pp/jobs/JOB_20200902_154540',
  'masterConfig': {'imageUri': 'gcr.io/etl-project-datahub/trainer_image:latest'}},
 'createTime': '2020-09-02T15:45:42Z',
 'startTime': '2020-09-02T15:45:43Z',
 'endTime': '2020-09-02T15:55:30Z',
 'state': 'SUC